#### Gauss-eliminasjon
Bruker vanlig Gauss-eliminasjon uten noen radbytter. Dvs. vi antar at $a_{ii} \neq 0 $. Merk at vi her må ta med vektoren $b$ inn i Gauss-eliminasjonen for at metoden skal virke. 


Ved back-substitusjonen antar man at inngangsparameteren $A$ er allerede Gauss-eliminert og er derfor en øvre triangulær matrise, vi kan kalle den $U$. Back-substitusjonen løser dermed $Ux=b$, der $b$ også må være Gauss-eliminert.  

In [1]:
import numpy as np

def gauss_elim(A, b):
    n = len(b)
    for j in range(n-1):
        for i in range(j+1,n):
            mult = A[i,j]/A[j,j]
            A[i,j:] = A[i,j:] - mult*A[j,j:]
            b[i] = b[i] - mult*b[j]
    return A, b

def back_substit_gauss(A, b):
    n = len(b)
    x = np.zeros(n)
    x[-1] = b[-1]/A[-1,-1]
    for i in range(n-2,-1,-1):
        x[i] = (b[i] - A[i,i+1:] @ x[i+1:])/A[i,i]
    return x

A = np.array([[1.0,2.0,-1.0],[2.0,1.0,-2.0],[-3.0,1.0,1.0]])
b = np.array([3.0,3.0,-6.0])
print("Bruker A=\n", A, "\nog b=\n", b)

A, b = gauss_elim(A,b)
print("\nGauss-eliminasjon gir")
print("A=\n", A,"\nb=\n",b)

print("\nLøsning på Ax=b")
x = back_substit_gauss(A,b)
print("x=\n",x)

Bruker A=
 [[ 1.  2. -1.]
 [ 2.  1. -2.]
 [-3.  1.  1.]] 
og b=
 [ 3.  3. -6.]

Gauss-eliminasjon gir
A=
 [[ 1.  2. -1.]
 [ 0. -3.  0.]
 [ 0.  0. -2.]] 
b=
 [ 3. -3. -4.]

Løsning på Ax=b
x=
 [3. 1. 2.]


#### LU-faktorisering med og uten pivotering

Finner her en nedre- og en øvretriangulær matrise slik at $A=LU$. LU-faktorisering fungerer på mange måter likt som Gauss-eliminasjon der man setter alle multiplikatorene, $l_{ij} = \frac{a_{ij}}{a_{jj}}$ for $i<j$, inn i den nedre triangulære matrisen $L$. $L$ har 1'ere på diagonalen. $U$ er den gjenstående Gauss-eliminerte matrisen fra $A$.  


Ved delvis pivotering faktoriserer man slik at man oppnår $PA=LU$, der $P$ er en permutasjonsmatrise. $L$ og $U$ er også her en nedre og øvre triangulær matrise, men ikke nødvendigvis de samme som ved vanlig LU-faktorisering. Ved denne metoden finner man for hver kolonne det største tallen i absoluttverdi blant de elementene som er på og under diagonalen. Hvis man finner et tall større enn det som er på diagonalen må man bytte slik at alltid det største står på digonalen. Skal man bytte må man bytte for $P$, $L$ og $U$ også for de samme radene. Etter byttet fortsetter man på samme vis som ved vanlig LU. Nå vil alle elementene i $L$ være mindre eller lik 1. $L$ har 1'ere på digonalen. 

I motsetning til Gauss-eliminasjon trenger man her kun å finne LU-faktoriseringen til $A$ én gang. Etter det kan man bruke $L$ og $U$ til backwards substitution med vektoren $b$. 

Delvis pivotering luker ut mulighetene for at man kan ha en null-pivot og får error.

In [5]:
def LU(A):
    U = A.copy()
    n = len(A)
    L = np.zeros((n,n))
    for j in range(n-1):
        for i in range(j+1,n):
            L[i,j] = U[i,j]/U[j,j]
            U[i,j:n] = U[i,j:n] - L[i,j]*U[j,j:n]
        L[j,j] = 1
    L[-1,-1] = 1
    return L, U

def switch(v1, v2):  #hjelpefunksjon
    t = v1.copy()
    v1 = v2
    v2 = t
    return v1, v2

def LU_pivot(A):
    U = A.copy()
    n = len(A)
    L = np.zeros((n,n))
    P = np.zeros((n,n))
    for k in range(n):
        P[k,k] = 1

    for j in range(n-1):
        high = abs(U[j,j])
        r_max = 0
        for l in range(j+1,n):
            if abs(U[l,j]) > high:
                high = abs(U[l,j])
                r_max = l
        if r_max != 0:
            P[r_max,:], P[j,:] = switch(P[r_max,:], P[j,:])
            U[r_max,:], U[j,:] = switch(U[r_max,:], U[j,:])
            L[r_max,:], L[j,:] = switch(L[r_max,:], L[j,:])

        for i in range(j+1,n):
            L[i,j] = U[i,j]/U[j,j]
            U[i,j:n] = U[i,j:n] - L[i,j]*U[j,j:n]
        L[j,j] = 1
    L[-1,-1] = 1
    return P, L, U

A = np.array([[1.0,2.0,-1.0],[2.0,1.0,-2.0],[-3.0,1.0,1.0]])

L, U = LU(A)
print("Vanlig LU-faktorisering gir")
print("L=\n", L,"\nU=\n",U)   #U er lik den gauss-eliminerte A'en
print("Tester om A=LU \n", L @ U ,"\n")

P, Lp, Up = LU_pivot(A)
print("LU-faktorisering med pivotering. U og L endrer seg her. Faktorene i L er mindre enn 1")
print("P=\n", P, "\nL=\n", Lp,"\nU=\n", Up)
print("Tester om PA=LU\n", P @ A ,"\n = \n", Lp @ Up)

Vanlig LU-faktorisering gir
L=
 [[ 1.          0.          0.        ]
 [ 2.          1.          0.        ]
 [-3.         -2.33333333  1.        ]] 
U=
 [[ 1.  2. -1.]
 [ 0. -3.  0.]
 [ 0.  0. -2.]]
Tester om A=LU 
 [[ 1.  2. -1.]
 [ 2.  1. -2.]
 [-3.  1.  1.]] 

LU-faktorisering med pivotering. U og L endrer seg her. Faktorene i L er mindre enn 1
P=
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]] 
L=
 [[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [-0.66666667  0.71428571  1.        ]] 
U=
 [[-3.          1.          1.        ]
 [ 0.          2.33333333 -0.66666667]
 [ 0.          0.         -0.85714286]]
Tester om PA=LU
 [[-3.  1.  1.]
 [ 1.  2. -1.]
 [ 2.  1. -2.]] 
 = 
 [[-3.  1.  1.]
 [ 1.  2. -1.]
 [ 2.  1. -2.]]


#### LU-faktorisering med pivoteringsvektor

I stedet for å bruke en $n x n$ permitasjonsmatrise $P$ kan man lage en permutasjonsvektor $P = [1,2,3...,n]^T$. I algoritmen nedenfor blir matrisen $A$ overskrivet til en ny matrise $LU$ av elementene i $L$ og $U$, da for å spare minneplass. 

For å aksessere elementer i $A$ bruker man f.eks. $A[P[k],k]$ for å få et element på diagonalen som er i henhold til $P$. Man kan også få hele rader ved $A[P,]$. Det er viktig å fortsette og aksessere den nye $LU$-matrisen i henhold til $P$ også etterpå når man skal bruke back-substitusjon.

Det er også laget en funksjon som returnerer to nye matriser $L$ og $U$ utifra $LU$-matrisen og permutasjonsvektoren $P$.



In [54]:
def LU_pivot_vec(B):
    A = np.array(B.copy())   #<-- denne kan fjernes hvis man vil overskrive A
    n = len(A)
    P = np.array([i for i in range(n)])

    for k in range(n-1):
        pivot = np.argmax(abs(A[P[k:],k]))
        if pivot != 0:
            P[k], P[pivot+k] = P[pivot+k], P[k]

        for i in range(k+1,n):
            A[P[i],k] = A[P[i],k]/A[P[k],k]
            A[P[i],k+1:n] = A[P[i],k+1:n] - A[P[i],k]*A[P[k],k+1:n]

    return P, A

def get_LU(LU, P): #Kan evt. bare lage en ny L og slette de nedre elementene i LU
    n = len(LU)
    L = np.zeros((n,n))
    U = np.zeros((n,n))
    for k in range(n):
        L[k,:k] = LU[P[k],:k]
        L[k,k] = 1
        U[k,k:] = LU[P[k],k:]
    return L, U

           
A = np.array([[1.0,2.0,-1.0],[2.0,1.0,-2.0],[-3.0,1.0,1.0]])
P, LU = LU_pivot_vec(A)
L, U = get_LU(LU, P)

print("LU(sammensatt) av L og U=\n", LU[P,])
print("PA=\n", A[P,])
print("LU=\n", L @ U)

LU(sammensatt) av L og U=
 [[-3.          1.          1.        ]
 [-0.33333333  2.33333333 -0.66666667]
 [-0.66666667  0.71428571 -0.85714286]]
PA=
 [[-3.  1.  1.]
 [ 1.  2. -1.]
 [ 2.  1. -2.]]
LU=
 [[-3.  1.  1.]
 [ 1.  2. -1.]
 [ 2.  1. -2.]]


#### Back-substitusjon LU
For LU-faktorisering skal man løse ligningen $LUx = b$ som man kan dele opp i to steg

1. Løs $Lc=b$ m.h.p $c$ og 
2. Løs $Ux=c$ m.h.p $x$.

Med delvis pivotering har man med permutasjonsmatrisen $P$ som tilfredstiller $PA = LU$. Ligningen man skal løse her blir da $LUx = Pb = \bar b$. Med delvis pivotering følger man stegene i backwards substitution som i stad, men med $Pb = \bar b$. Ved bruk av permutasjonsvektor $P$ kan man finne $\bar b$ ved hjelp av $\bar b = b[P]$. 

Merk at steg 2. er nøyaktig det samme som back-substitusjon ved Gauss-eliminasjon med $A'=U$ og $b'=c$ ( '=gauss-eliminert).

In [3]:
def back_substit_LU(L, U, b): #b = Pb for tilfelle med pivotering 
    n = len(b)
    c = np.zeros(n)
    c[0] = b[0]/L[0,0]
    for i in range(1,n):
        c[i] = (b[i] - L[i,:i] @ c[:i])/L[i,i] #Trenger egt. ikke å dele på l_ii siden den er 1 her.
    
    x = np.zeros(n)     #Koden under er det samme som x = back_substit_gauss(U,c) 
    x[-1] = c[-1]/U[-1,-1]
    for i in range(n-2,-1,-1):
        x[i] = (c[i] - U[i,i+1:] @ x[i+1:])/U[i,i]
    return x


A = np.array([[1.0,2.0,-1.0],[2.0,1.0,-2.0],[-3.0,1.0,1.0]])
b = np.array([3.0,3.0,-6.0])

L, U = LU(A)
x = back_substit_LU(L, U, b)
A = L @ U
print("Løsning til Ax=b, x=\n", x)
print("Sjekker at LUx=b\n", L @ U @ x,"=", b)

P, Lp, Up = LU_pivot(A)
b_bar = P @ b
x = back_substit_LU(Lp, Up, b_bar)
print("\nLøsning til Ax=b, x=\n", x)
print("Sjekker at LUx=Pb\n", Lp @ Up @ x,"=", b_bar)

Løsning til Ax=b, x=
 [3. 1. 2.]
Sjekker at LUx=b
 [ 3.  3. -6.] = [ 3.  3. -6.]

Løsning til Ax=b, x=
 [3. 1. 2.]
Sjekker at LUx=Pb
 [-6.  3.  3.] = [-6.  3.  3.]


#### Cholesky-faktorisering

Faktoriserer $A$ slik at $A=LL^{T}$. Denne metoden gjelder kun for matriser som er symmetrisk positiv-definitt (SPD). Her har $L$ generelt ikke 1'ere på diagonalen.

Ved back-substitusjon for Cholesky løser man likningen $LL^{T}x=b$ ved to steg

1. $Lc = b$ m.h.p. c
2. $L^{T}x = c$ m.h.p. x

Vi ser at stegene er nesten helt like som for back-substitusjon med LU. Hvis man setter back_substit_LU($L$,$L^{T}$,$b$) får man riktig svar. Nedenfor er en funksjon som kun tar inn $L$ og $b$ som parametere og, uten å bruke andre funksjoner eller transponere $L$, finner en løsning $x$. Merk at her er det kun steg 2. som blir annerledes.

In [4]:
def cholesky(A):
    n=A.shape[1]
    L=np.zeros((n,n))
    for i in range(n):
        for j in range(i):
            L[i,j]=(A[i,j] - L[i,:j] @ L[j,:j])/L[j,j]
        L[i,i] = np.sqrt(A[i,i]-L[i,:i] @ L[i,:i])
    
    return L

def back_substit_cholesky(L,b):  #Er det samme som back_substit_LU(L,L^T,b), hvis man husker å dele på l_ii
    n = len(b)
    c=np.zeros(n)
    c[0]=b[0]/L[0,0];   #L kan her ha noe annet enn 1 på diagonalen
    for i in range(1,n):
        c[i] = (b[i] - L[i,:i] @ c[:i])/L[i,i]
    
    x=np.zeros(n)
    x[-1] = c[-1]/L[-1,-1]
    for i in range(n-2,-1,-1):
        x[i]=(c[i] - L[i+1:,i] @ x[i+1:])/L[i,i]
    return x

A = np.array([[1,2,3],[2,5,4],[3,4,14]])
b = np.array([-2,-8,3])
L = cholesky(A)
x = back_substit_cholesky(L,b)

print("Løsning på Ax=b, x=\n", x)
print("Tester om LL^T = A\n", L @ np.transpose(L), "\n=\n", A)

Løsning på Ax=b, x=
 [-1. -2.  1.]
Tester om LL^T = A
 [[ 1.  2.  3.]
 [ 2.  5.  4.]
 [ 3.  4. 14.]] 
=
 [[ 1  2  3]
 [ 2  5  4]
 [ 3  4 14]]
